<a href="https://colab.research.google.com/github/chrisoyer/thinkful_notes/blob/master/Unit_4_Spark/Amazon_Kindle_Reviews__Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description:
Challenge: Amazon Reviews Sentiment Analysis

### Initialization

In [0]:
#environment
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_CLASSPATH"] = '/content/spark-2.4.4-bin-hadoop2.7'

In [3]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 124kB/s 
     |████████████████████████████████| 204kB 53.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=af73875db0e03f9cf27b9aa9bd6f5f5c994b807b905d8fbd45eaea4b95a9d94a
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Spark NLP') \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.2.2") \
    .getOrCreate()

In [0]:
source_url = r"http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz"

I will be using gDrive file location instead of downloading the source file.

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
local_souce_file = r"/content/gdrive/My Drive/thinkful/colab_datasets/reviews_Grocery_and_Gourmet_Food_5.json.gz"
SPARK_URL = "local[*]"
APP_NAME  = "amazon_food_reviews"

In [9]:
from pyspark import SparkContext
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
import pyspark.sql.functions as F

!pip install spark-nlp
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import LightPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher
print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
2.2.2
Apache Spark version


'2.4.4'

In [10]:
sparknlp.start()
pipeline = PretrainedPipeline(name='analyze_sentiment', lang='en')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]


In [0]:
#spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
# not working# !gzip -dkc $local_source_file > /content/reviews
reviews = r'/content/gdrive/My Drive/thinkful/colab_datasets/reviews_Grocery_and_Gourmet_Food_5.json'
reviews_df = spark.read.options(inferschema = "true").json(reviews)

In [12]:
reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [13]:
reviews_df.show(5)

+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|   reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|616719923X| [0, 0]|    4.0|Just another flav...| 06 1, 2013|A1VEELTKS8NLZB|Amazon Customer|          Good Taste|    1370044800|
|616719923X| [0, 1]|    3.0|I bought this on ...|05 19, 2014|A14R9XMZVJ6INB|        amf0001|3.5 stars,  sadly...|    1400457600|
|616719923X| [3, 4]|    4.0|Really good. Grea...| 10 8, 2013|A27IQHDZFQFNGG|        Caitlin|                Yum!|    1381190400|
|616719923X| [0, 0]|    5.0|I had never had i...|05 20, 2013|A31QY5TASILE89|   DebraDownSth|Unexpected flavor...|    1369008000|
|616719923X| [1, 2]|    4.0|I've been looking...|05 26, 2013|A2LWK003FFMCI5|       Diana X.|Not a

In [14]:
reviews_df.select('overall').describe().show()

+-------+------------------+
|summary|           overall|
+-------+------------------+
|  count|            151254|
|   mean| 4.243041506340329|
| stddev|1.0900026138973262|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



In [0]:
reviews_df = reviews_df.withColumn('sentiment_label', F.when(reviews_df["overall"] >= 4, 'Positive').otherwise('Negative'))

In [16]:
reviews_df.show(10)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+---------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|sentiment_label|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+---------------+
|616719923X| [0, 0]|    4.0|Just another flav...| 06 1, 2013|A1VEELTKS8NLZB|     Amazon Customer|          Good Taste|    1370044800|       Positive|
|616719923X| [0, 1]|    3.0|I bought this on ...|05 19, 2014|A14R9XMZVJ6INB|             amf0001|3.5 stars,  sadly...|    1400457600|       Negative|
|616719923X| [3, 4]|    4.0|Really good. Grea...| 10 8, 2013|A27IQHDZFQFNGG|             Caitlin|                Yum!|    1381190400|       Positive|
|616719923X| [0, 0]|    5.0|I had never had i...|05 20, 2013|A31QY5TASILE89|        DebraDownSth|Une

In [0]:
reviews_df = reviews_df.na.drop()

# Spark NLP Sentiment Analysis


Using example from jonsnow sparknlp


In [0]:
# use document assemble which puts data in annotaed form
document_assembler = DocumentAssembler() \
                      .setInputCol("reviewText") \
                      .setOutputCol("review_document")

In [19]:
assembled = document_assembler.transform(reviews_df)
assembled.select('review_document').take(5)

[Row(review_document=[Row(annotatorType='document', begin=0, end=161, result='Just another flavor of Kit Kat but the taste is unique and a bit different.  The only thing that is bothersome is the price.  I thought it was a bit expensive....', metadata={'sentence': '0'}, embeddings=[], sentence_embeddings=[])]),
 Row(review_document=[Row(annotatorType='document', begin=0, end=582, result="I bought this on impulse and it comes from Japan,  which amused my family,  all those weird stamps and markings on the package. So that was fun.  It said it would take about a month to arrive and it did take that long.  I was hoping for a more interesting taste but to our family,  it just tasted a bit less flavorful or weaker than the standard milk chocolate kit kat.  The green tea flavor was too subtle for the sugar and it just tasted sweet. The wafers were very crispy, and that was good,  but it tasted a bit anemic to us.I'm happy I bought it, but don't need to buy it again.", metadata={'sentence': '

In [20]:
#detect sentences
sentence_finder = SentenceDetector() \
    .setExplodeSentences(False) \
    .setInputCols("review_document") \
    .setOutputCol("sentence") 
sentence_data = sentence_finder.transform(assembled)
sentence_data.select("sentence").limit(5).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sentence                                                  

In [21]:
first_obs = sentence_data.select('sentence') \
      .limit(1)
first_obs_df = first_obs.select('sentence', F.explode(first_obs.sentence).alias('_sentence'))
first_obs_df.toPandas()[['_sentence']]

,_sentence
0,"(document, 0, 74, Just another flavor of Kit K..."
1,"(document, 77, 123, The only thing that is bot..."
2,"(document, 126, 158, I thought it was a bit ex..."
3,"(document, 159, 159, ., {'sentence': '3'}, [],..."
4,"(document, 160, 160, ., {'sentence': '4'}, [],..."
5,"(document, 161, 161, ., {'sentence': '5'}, [],..."


In [0]:
#Tokenize
tokenizer = Tokenizer() \
              .setInputCols(['sentence']) \
              .setOutputCol('token')
token_data = tokenizer.fit(sentence_data).transform(sentence_data)

In [0]:
#Normalize
normalizer = Normalizer() \
                .setInputCols(["token"]) \
                .setOutputCol('normed_token')


In [24]:
# -N new only
# -P set directory
! wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/spell/words.txt -P /tmp

--2019-10-15 19:44:49--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/spell/words.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.1.3
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.1.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4862966 (4.6M) [text/plain]
Saving to: ‘/tmp/words.txt’

words.txt           100%[===================>]   4.64M  --.-KB/s    in 0.1s    

2019-10-15 19:44:49 (45.9 MB/s) - ‘/tmp/words.txt’ saved [4862966/4862966]



In [0]:
#Check Spelling
spell_check = NorvigSweetingApproach() \
                .setInputCols(['normed_token']) \
                .setOutputCol('spell_checked') \
                .setDictionary("/tmp/words.txt")

In [0]:
#sentiment
sentiment_analyzer = ViveknSentimentApproach() \
                      .setInputCols(['spell_checked', 'sentence']) \
                      .setOutputCol('sentiment') \
                      .setPruneCorpus(0) \
                      .setSentimentCol('sentiment_label')
                      

In [0]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeMetadata(False)

In [0]:
sentiment_pipe = Pipeline(stages=[
                                  document_assembler,
                                  sentence_finder,
                                  tokenizer,
                                  normalizer,
                                  spell_check,
                                  sentiment_analyzer,
                                  finisher
                                  ])

In [31]:
review_sentiment_model = sentiment_pipe.fit(reviews_df)

IllegalArgumentException: ignored

In [0]:
review_sentiment_data = review_sentiment_model.transform(reviews_df)

In [0]:
review_sentiment_data.show(n=5, truncate=False)

In [0]:
sentiment_pipe2 = LightPipeline(sentiment_pipe)

In [34]:
prepipe = PretrainedPipeline(name='analyze_sentiment')

result = prepipe.annotate(target=reviews_df, column="reviewText")

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]


In [39]:
spark.createDataFrame(result.select("sentiment", "sentiment_label").take(5))

ValueError: ignored

In [36]:
sc = spark.sparkContext
results = sc.parallelize(result.select("sentiment", "sentiment_label").take(5)) \
          #.explode().map(lambda line: tuple([int(x) for x in line])) \
          

results.collect()[0][0]

[Row(annotatorType='sentiment', begin=0, end=74, result='positive', metadata={'confidence': '0.5257'}, embeddings=[], sentence_embeddings=[]),
 Row(annotatorType='sentiment', begin=77, end=123, result='negative', metadata={'confidence': '0.4548'}, embeddings=[], sentence_embeddings=[]),
 Row(annotatorType='sentiment', begin=126, end=158, result='positive', metadata={'confidence': '0.5447'}, embeddings=[], sentence_embeddings=[]),
 Row(annotatorType='sentiment', begin=159, end=159, result='negative', metadata={'confidence': '0.5140'}, embeddings=[], sentence_embeddings=[]),
 Row(annotatorType='sentiment', begin=160, end=160, result='negative', metadata={'confidence': '0.5140'}, embeddings=[], sentence_embeddings=[]),
 Row(annotatorType='sentiment', begin=161, end=161, result='negative', metadata={'confidence': '0.5140'}, embeddings=[], sentence_embeddings=[])]

In [0]:
spark.createDataFrame(results).show(5)